# Check for negative controls in all the plates

In [1]:
import os
import os.path as osp

import pandas as pd
import seaborn as sns
import yaml

In [2]:
metadata_path = "../cpjump1/jump/metadata"
load_data_path = "../cpjump1/jump/load_data"

In [3]:
os.listdir(metadata_path)

['compound.csv.gz',
 'crispr.csv.gz',
 'microscope_config.csv',
 'microscope_filter.csv',
 'orf.csv.gz',
 'plate.csv.gz',
 'README.md',
 'well.csv.gz',
 'compound.csv',
 'crispr.csv',
 'orf.csv',
 'plate.csv',
 'well.csv',
 'complete_metadata.csv',
 'resolution.csv',
 'JUMP-Target-1_compound_metadata.tsv',
 'JUMP-Target-1_compound_platemap.tsv',
 'JUMP-Target-1_crispr_metadata.tsv',
 'JUMP-Target-1_crispr_platemap.tsv',
 'JUMP-Target-1_orf_metadata.tsv',
 'JUMP-Target-1_orf_platemap.tsv',
 'JUMP-Target-2_compound_metadata.tsv',
 'JUMP-Target-2_compound_platemap.tsv',
 'JUMP-MOA_compound_metadata.tsv']

In [5]:
os.listdir("../cpjump1/jump/load_data")

['load_data_with_metadata', 'load_data_with_samples', 'final']

In [18]:
with open(
    "/mnt/2547d4d7-6732-4154-b0e1-17b0c1e0c565/Document-2/Projet2/Stage/workspace/jump_download/conf/filters/filters_default.yaml",
) as f:
    filters = yaml.load(f, Loader=yaml.FullLoader)

In [19]:
filters

{'plate_types_to_keep': ['COMPOUND', 'ORF', 'CRISPR', 'TARGET1', 'TARGET2'],
 'sources_to_exclude': ['source_7'],
 'compound_number_of_poscon_to_keep_per_well': 4,
 'compound_number_of_negcon_to_keep_per_well': 3,
 'compound_number_of_trt_to_keep_per_well': 6,
 'orf_number_of_poscon_to_keep_per_well': 4,
 'orf_number_of_negcon_to_keep_per_well': 3,
 'orf_number_of_trt_to_keep_per_well': 6,
 'crispr_number_of_poscon_to_keep_per_well': 4,
 'crispr_number_of_negcon_to_keep_per_well': 3,
 'crispr_number_of_trt_to_keep_per_well': 6,
 'target_number_of_poscon_to_keep_per_well': 4,
 'target_number_of_negcon_to_keep_per_well': 3,
 'target_number_of_trt_to_keep_per_well': 6,
 'compound_negative_controls': ['JCP2022_033924'],
 'compound_positive_controls': ['JCP2022_037716',
  'JCP2022_064022',
  'JCP2022_025848',
  'JCP2022_050797',
  'JCP2022_046054',
  'JCP2022_012818',
  'JCP2022_085227',
  'JCP2022_035095',
  'JCP2022_033954'],
 'compound_pert_to_drop': [],
 'orf_negative_controls': ['JCP20

In [10]:
plates = pd.read_csv(osp.join(metadata_path, "plate.csv"))

In [11]:
wells = pd.read_csv(osp.join(metadata_path, "well.csv"))

In [12]:
paw = pd.merge(plates, wells, on=["Metadata_Source", "Metadata_Plate"], how="left")

In [11]:
paw.groupby(["Metadata_Source", "Metadata_PlateType"]).agg(
    Num_plate=("Metadata_Plate", "nunique"),
)

Num_plate
Metadata_Source Metadata_PlateType           
source_1        COMPOUND                   51
                COMPOUND_EMPTY              4
source_10       COMPOUND                  208
                DMSO                        8
                TARGET2                     6
source_11       COMPOUND                  173
                TARGET2                     7
source_13       CRISPR                    148
                TARGET2                     6
source_2        COMPOUND                  219
                TARGET2                    10
source_3        COMPOUND                  237
                COMPOUND_EMPTY              3
                DMSO                       38
                TARGET2                    25
source_4        ORF                       251
                TARGET1                     4
                TARGET2                    22
source_5        COMPOUND                  197
                DMSO                       31
                POSCON8                     8
                TARGET2                    24
source_6        COMPOUND                  225
                TARGET2                    21
source_7        COMPOUND                  121
                TARGET2                     7
source_8        COMPOUND                  199
                COMPOUND_EMPTY             13
                TARGET2                     4
source_9        COMPOUND                   99
                TARGET2                     9

In [8]:
paw.Metadata_PlateType.value_counts()

Metadata_PlateType
COMPOUND          832295
ORF                96377
TARGET2            64485
CRISPR             56832
DMSO               29566
COMPOUND_EMPTY     11914
POSCON8             3072
TARGET1             1533
Name: count, dtype: int64

In [16]:
meta = pd.read_csv(osp.join(metadata_path, "complete_metadata.csv"))

/tmp/ipykernel_8748/2571030615.py:1: DtypeWarning: Columns (2,8,9,10,11,12,13,14,15,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(osp.join(metadata_path, "complete_metadata.csv"))


In [43]:
filters["orf_positive_controls"]

['JCP2022_915132',
 'JCP2022_037716',
 'JCP2022_035095',
 'JCP2022_050797',
 'JCP2022_012818']

In [48]:
meta["Metadata_JCP2022"].isin(filters["orf_positive_controls"]).sum()

38856

In [51]:
(meta["Metadata_JCP2022"].isin(filters["orf_positive_controls"]) & meta["Metadata_PlateType"] == "ORF").sum()

0

In [20]:
well_types = meta.assign(
    compound_negative_controls=lambda df: df["Metadata_JCP2022"].isin(filters["compound_negative_controls"])
    & ~df["Metadata_PlateType"].isin(["ORF", "CRISPR", "TARGET1", "TARGET2"]),
    compound_positive_controls=lambda df: df["Metadata_JCP2022"].isin(filters["compound_positive_controls"])
    & ~df["Metadata_PlateType"].isin(["ORF", "CRISPR", "TARGET1", "TARGET2"]),
    compound_to_drop=lambda df: df["Metadata_JCP2022"].isin(filters["compound_pert_to_drop"])
    & ~df["Metadata_PlateType"].isin(["ORF", "CRISPR", "TARGET1", "TARGET2"]),
    orf_negative_controls=lambda df: df["Metadata_JCP2022"].isin(filters["orf_negative_controls"])
    & df["Metadata_PlateType"]
    == "ORF",
    orf_positive_controls=lambda df: df["Metadata_JCP2022"].isin(filters["orf_positive_controls"])
    & df["Metadata_PlateType"]
    == "ORF",
    orf_to_drop=lambda df: df["Metadata_JCP2022"].isin(filters["orf_pert_to_drop"]) & df["Metadata_PlateType"] == "ORF",
    crispr_positive_controls=lambda df: df["Metadata_JCP2022"].isin(filters["crispr_positive_controls"])
    & df["Metadata_PlateType"]
    == "CRISPR",
    crispr_negative_controls=lambda df: df["Metadata_JCP2022"].isin(filters["crispr_negative_controls"])
    & df["Metadata_PlateType"]
    == "CRISPR",
    crispr_to_drop=lambda df: df["Metadata_JCP2022"].isin(filters["crispr_pert_to_drop"]) & df["Metadata_PlateType"]
    == "CRISPR",
    target_negative_controls=lambda df: df["Metadata_JCP2022"].isin(filters["target_negative_controls"])
    & df["Metadata_PlateType"].isin(["TARGET1", "TARGET2"]),
    target_positive_controls=lambda df: df["Metadata_JCP2022"].isin(filters["target_positive_controls"])
    & df["Metadata_PlateType"].isin(["TARGET1", "TARGET2"]),
    target_to_drop=lambda df: df["Metadata_JCP2022"].isin(filters["target_pert_to_drop"])
    & df["Metadata_PlateType"].isin(["TARGET1", "TARGET2"]),
)

In [54]:
def col_to_str(x):
    if x["compound_negative_controls"]:
        return "compound_neg"
    elif x["compound_positive_controls"]:
        return "compound_pos"
    elif x["compound_to_drop"]:
        return "compound_drop"
    elif x["orf_negative_controls"]:
        return "orf_neg"
    elif x["orf_positive_controls"]:
        return "orf_pos"
    elif x["orf_to_drop"]:
        return "orf_drop"
    elif x["crispr_negative_controls"]:
        return "crispr_neg"
    elif x["crispr_positive_controls"]:
        return "crispr_pos"
    elif x["crispr_to_drop"]:
        return "crispr_drop"
    elif x["target_negative_controls"]:
        return "target_neg"
    elif x["target_positive_controls"]:
        return "target_pos"
    elif x["target_to_drop"]:
        return "target_drop"
    else:
        if x["Metadata_PlateType"] == "COMPOUND":
            return "compound_trt"
        elif x["Metadata_PlateType"] == "CRISPR":
            return "crispr_trt"
        elif x["Metadata_PlateType"] == "ORF":
            return "orf_trt"
        elif x["Metadata_PlateType"] in ["TARGET1", "TARGET2"]:
            return "target_trt"
        else:
            return "other"

In [42]:
well_types.orf_negative_controls.sum()

0

In [55]:
trt = well_types.apply(col_to_str, axis=1)

In [57]:
well_types["trt"] = trt

In [56]:
trt.value_counts()

compound_trt    679106
compound_neg    126739
orf_trt          96377
compound_pos     68442
crispr_trt       56832
target_trt       54949
target_neg       11069
other             2560
Name: count, dtype: int64

In [36]:
q = well_types.groupby(["Metadata_Source", "Metadata_PlateType", "Metadata_Plate"]).compound_negative_controls.sum()
q

Metadata_Source  Metadata_PlateType
source_1         COMPOUND               9217
                 COMPOUND_EMPTY         5518
source_10        COMPOUND               6560
                 DMSO                   3071
                 TARGET2                   0
source_11        COMPOUND               7587
                 TARGET2                   0
source_13        CRISPR                    0
                 TARGET2                   0
source_2         COMPOUND              10877
                 TARGET2                   0
source_3         COMPOUND               7574
                 COMPOUND_EMPTY           96
                 DMSO                  14591
                 TARGET2                   0
source_4         ORF                       0
                 TARGET1                   0
                 TARGET2                   0
source_5         COMPOUND               6304
                 DMSO                  11904
                 POSCON8                   0
                 TA

In [7]:
df = pd.read_parquet(osp.join(load_data_path, "final"))

In [58]:
filtered_meta = df[["Metadata_Source", "Metadata_Batch", "Metadata_Plate", "Metadata_Well"]].drop_duplicates()

In [59]:
well_types.columns

Index(['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate',
       'Metadata_PlateType', 'Metadata_Well', 'Metadata_JCP2022',
       'Metadata_InChIKey', 'Metadata_InChI', 'Metadata_broad_sample',
       'Metadata_Name', 'Metadata_Vector', 'Metadata_Transcript',
       'Metadata_Symbol_x', 'Metadata_NCBI_Gene_ID_x', 'Metadata_Taxon_ID',
       'Metadata_Gene_Description', 'Metadata_Prot_Match',
       'Metadata_Insert_Length', 'Metadata_pert_type',
       'Metadata_NCBI_Gene_ID_y', 'Metadata_Symbol_y',
       'Metadata_Microscope_Name', 'Metadata_Widefield_vs_Confocal',
       'Metadata_Excitation_Type', 'Metadata_Objective_NA',
       'Metadata_N_Brightfield_Planes_Min',
       'Metadata_N_Brightfield_Planes_Max',
       'Metadata_Distance_Between_Z_Microns', 'Metadata_Sites_Per_Well',
       'Metadata_Filter_Configuration', 'compound_negative_controls',
       'compound_positive_controls', 'compound_to_drop',
       'orf_negative_controls', 'orf_positive_controls', 'orf_to_drop',
 

In [60]:
cols_to_keep = [
    "Metadata_Source",
    "Metadata_Batch",
    "Metadata_Plate",
    "Metadata_PlateType",
    "Metadata_Well",
    "Metadata_JCP2022",
    "Metadata_InChIKey",
    "Metadata_InChI",
    "Metadata_Sites_Per_Well",
    "trt",
]

In [61]:
filtered_meta = well_types.merge(filtered_meta)[cols_to_keep]

In [64]:
filtered_meta.trt.value_counts()

trt
compound_trt    545924
orf_trt          86935
compound_neg     65105
crispr_trt       56832
compound_pos     51123
target_trt       45992
target_neg        9251
Name: count, dtype: int64

In [65]:
filtered_meta

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_PlateType,Metadata_Well,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI,Metadata_Sites_Per_Well,trt
0,source_1,Batch1_20221004,UL001641,COMPOUND,A02,JCP2022_033924,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,4,compound_neg
1,source_1,Batch1_20221004,UL001641,COMPOUND,A03,JCP2022_085227,SRVFFFJZQVENJC-UHFFFAOYSA-N,InChI=1S/C17H30N2O5/c1-6-23-17(22)14-13(24-14)...,4,compound_pos
2,source_1,Batch1_20221004,UL001641,COMPOUND,A04,JCP2022_033924,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,4,compound_neg
3,source_1,Batch1_20221004,UL001641,COMPOUND,A05,JCP2022_036592,IPPYTNWGGOIMDZ-UHFFFAOYSA-N,InChI=1S/C17H12ClF3N4O2/c1-24(15-13(18)6-9(7-2...,4,compound_trt
4,source_1,Batch1_20221004,UL001641,COMPOUND,A06,JCP2022_071885,PYZMXVUWLLQNEP-UHFFFAOYSA-N,InChI=1S/C10H7ClN4/c11-9-3-1-2-8(4-9)6-15-7-13...,4,compound_trt
...,...,...,...,...,...,...,...,...,...,...
861157,source_9,20211103-Run16,GR00004421,COMPOUND,Z44,JCP2022_999999,NaN,NaN,4,compound_trt
861158,source_9,20211103-Run16,GR00004421,COMPOUND,Z45,JCP2022_999999,NaN,NaN,4,compound_trt
861159,source_9,20211103-Run16,GR00004421,COMPOUND,Z46,JCP2022_999999,NaN,NaN,4,compound_trt
861160,source_9,20211103-Run16,GR00004421,COMPOUND,Z47,JCP2022_033924,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,4,compound_neg


In [67]:
filtered_meta.to_csv(osp.join(metadata_path, "local_metadata.csv"), index=False)

In [31]:
q = (
    filtered_meta.assign(is_negative=lambda x: x.Metadata_JCP2022 == "JCP2022_033924")
    .groupby(["Metadata_Source", "Metadata_PlateType"])
    .agg(
        has_negatives=("is_negative", "sum"),
    )
)
q

has_negatives
Metadata_Source Metadata_PlateType               
source_1        COMPOUND                     9217
source_10       COMPOUND                     6533
                TARGET2                       384
source_11       COMPOUND                     7555
                TARGET2                       448
source_13       CRISPR                       1128
                TARGET2                       414
source_2        COMPOUND                    10877
                TARGET2                       649
source_3        COMPOUND                     7574
                TARGET2                      1600
source_4        ORF                             0
                TARGET1                       256
                TARGET2                      1408
source_5        COMPOUND                     6304
                TARGET2                      1536
source_6        COMPOUND                    12832
                TARGET2                      1366
source_8        COMPOUND                     7776
                TARGET2                       256
source_9        COMPOUND                    12672
                TARGET2                      2304

has_negatives
Metadata_Source Metadata_Batch  Metadata_PlateType               
source_1        Batch1_20221004 COMPOUND                      768
                Batch2_20221006 COMPOUND                     1696
                Batch3_20221010 COMPOUND                     1605
                Batch4_20221012 COMPOUND                     1019
                Batch5_20221030 COMPOUND                     1473
...                                                           ...
source_9        20211013-Run14  TARGET2                       256
                20211102-Run15  COMPOUND                     1280
                                TARGET2                       256
                20211103-Run16  COMPOUND                     2048
                                TARGET2                       256

[236 rows x 1 columns]

In [14]:
df

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_Site,FileName_OrigAGP,FileName_OrigDNA,FileName_OrigER,FileName_OrigMito,FileName_OrigRNA
0,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A01,1,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
1,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A01,3,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
2,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A01,6,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
3,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A02,1,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
4,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A02,2,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...,/projects/cpjump3/jump/images/source_10/2021_0...
...,...,...,...,...,...,...,...,...,...,...
5121195,source_9,20211103-Run16,GR00004421,Z47,3,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...
5121196,source_9,20211103-Run16,GR00004421,Z47,4,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...
5121197,source_9,20211103-Run16,GR00004421,Z48,1,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...
5121198,source_9,20211103-Run16,GR00004421,Z48,2,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...,/projects/cpjump1/jump/images/source_9/2021110...


In [128]:
df.merge(paw).groupby(["Metadata_Source", "Metadata_PlateType"]).FileName_OrigAGP.nunique()

Metadata_Source  Metadata_PlateType
source_1         COMPOUND              296092
source_10        COMPOUND              445602
                 TARGET2                12660
source_11        COMPOUND              360873
                 TARGET2                14758
source_13        CRISPR                305012
                 TARGET2                12582
source_2         COMPOUND              462180
                 TARGET2                20964
source_3         COMPOUND              507686
                 TARGET2                52794
source_4         ORF                   504138
                 TARGET1                 8430
                 TARGET2                46428
source_5         COMPOUND              422356
                 TARGET2                50676
source_6         COMPOUND              472621
                 TARGET2                44286
source_8         COMPOUND              424116
                 TARGET2                 8448
source_9         COMPOUND              59551

In [130]:
52986 / 595512

0.0889755370168863

In [48]:
unique_wells = df[["Metadata_Source", "Metadata_Batch", "Metadata_Plate", "Metadata_Well"]].drop_duplicates()

In [49]:
unique_wells

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well
0,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A01
3,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A02
7,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A03
13,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A04
19,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A05
...,...,...,...,...
5121181,source_9,20211103-Run16,GR00004421,Z44
5121185,source_9,20211103-Run16,GR00004421,Z45
5121189,source_9,20211103-Run16,GR00004421,Z46
5121193,source_9,20211103-Run16,GR00004421,Z47


In [50]:
df_wc = pd.merge(unique_wells, well_types, on=["Metadata_Source", "Metadata_Batch", "Metadata_Plate", "Metadata_Well"])

In [54]:
df_wc

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_PlateType,Metadata_JCP2022,compound_negative_controls,compound_positive_controls,compound_to_drop,orf_negative_controls,orf_positive_controls,orf_to_drop,crispr_positive_controls,crispr_negative_controls,crispr_to_drop,target_negative_controls,target_positive_controls,target_to_drop
0,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A01,COMPOUND,JCP2022_085227,False,True,False,False,False,False,False,False,False,False,False,False
1,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A02,COMPOUND,JCP2022_033924,True,False,False,False,False,False,False,False,False,False,False,False
2,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A03,COMPOUND,JCP2022_056163,False,False,False,False,False,False,False,False,False,False,False,False
3,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A04,COMPOUND,JCP2022_054175,False,False,False,False,False,False,False,False,False,False,False,False
4,source_10,2021_05_31_U2OS_48_hr_run1,Dest210531-152149,A05,COMPOUND,JCP2022_999999,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992229,source_9,20211103-Run16,GR00004421,Z44,COMPOUND,JCP2022_999999,False,False,False,False,False,False,False,False,False,False,False,False
992230,source_9,20211103-Run16,GR00004421,Z45,COMPOUND,JCP2022_999999,False,False,False,False,False,False,False,False,False,False,False,False
992231,source_9,20211103-Run16,GR00004421,Z46,COMPOUND,JCP2022_999999,False,False,False,False,False,False,False,False,False,False,False,False
992232,source_9,20211103-Run16,GR00004421,Z47,COMPOUND,JCP2022_033924,True,False,False,False,False,False,False,False,False,False,False,False


In [88]:
q = (
    df_wc.groupby(["Metadata_Source", "Metadata_Batch", "Metadata_PlateType", "Metadata_Plate"])
    .agg(
        has_neg_compound=("compound_negative_controls", "sum"),
        has_neg_orf=("orf_negative_controls", "sum"),
        has_neg_crispr=("crispr_negative_controls", "sum"),
        has_neg_target=("target_negative_controls", "sum"),
        num_wells=("Metadata_Well", "nunique"),
    )
    .reset_index()
    .assign(has_neg=lambda x: x.has_neg_compound + x.has_neg_orf + x.has_neg_crispr + x.has_neg_target)
)
qq

,Metadata_Source,Metadata_Batch,Metadata_PlateType,Metadata_Plate,has_neg_compound,has_neg_orf,has_neg_crispr,has_neg_target,num_wells,has_neg
0,source_1,Batch1_20221004,COMPOUND,UL001641,128,0,0,0,1472,128
1,source_1,Batch1_20221004,COMPOUND,UL001643,128,0,0,0,1472,128
2,source_1,Batch1_20221004,COMPOUND,UL001645,128,0,0,0,1472,128
3,source_1,Batch1_20221004,COMPOUND,UL001651,128,0,0,0,1472,128
4,source_1,Batch1_20221004,COMPOUND,UL001653,128,0,0,0,1472,128
...,...,...,...,...,...,...,...,...,...,...
2125,source_9,20211103-Run16,COMPOUND,GR00004418,128,0,0,0,1536,128
2126,source_9,20211103-Run16,COMPOUND,GR00004419,128,0,0,0,1536,128
2127,source_9,20211103-Run16,COMPOUND,GR00004420,128,0,0,0,1536,128
2128,source_9,20211103-Run16,COMPOUND,GR00004421,128,0,0,0,1536,128


In [89]:
qq = (
    well_types.groupby(["Metadata_Source", "Metadata_Batch", "Metadata_PlateType", "Metadata_Plate"])
    .agg(
        has_neg_compound=("compound_negative_controls", "sum"),
        has_neg_orf=("orf_negative_controls", "sum"),
        has_neg_crispr=("crispr_negative_controls", "sum"),
        has_neg_target=("target_negative_controls", "sum"),
        num_wells=("Metadata_Well", "nunique"),
    )
    .reset_index()
    .assign(has_neg=lambda x: x.has_neg_compound + x.has_neg_orf + x.has_neg_crispr + x.has_neg_target)
)
qq

,Metadata_Source,Metadata_Batch,Metadata_PlateType,Metadata_Plate,has_neg_compound,has_neg_orf,has_neg_crispr,has_neg_target,num_wells,has_neg
0,source_1,Batch1_20221004,COMPOUND,UL001641,128,0,0,0,1472,128
1,source_1,Batch1_20221004,COMPOUND,UL001643,128,0,0,0,1472,128
2,source_1,Batch1_20221004,COMPOUND,UL001645,128,0,0,0,1472,128
3,source_1,Batch1_20221004,COMPOUND,UL001651,128,0,0,0,1472,128
4,source_1,Batch1_20221004,COMPOUND,UL001653,128,0,0,0,1472,128
...,...,...,...,...,...,...,...,...,...,...
2373,source_9,20211103-Run16,COMPOUND,GR00004418,128,0,0,0,1536,128
2374,source_9,20211103-Run16,COMPOUND,GR00004419,128,0,0,0,1536,128
2375,source_9,20211103-Run16,COMPOUND,GR00004420,128,0,0,0,1536,128
2376,source_9,20211103-Run16,COMPOUND,GR00004421,128,0,0,0,1536,128


In [98]:
(q.has_neg_compound > 0).sum() / q.shape[0]

0.7530516431924883

In [103]:
(qqq.has_neg_compound > 0).sum() / qqq.shape[0]

0.9981331673926571

In [101]:
qqq = q[q["Metadata_PlateType"] == "COMPOUND"]

In [114]:
(qqq.has_neg_compound < 20).sum()

6

In [63]:
q["Metadata_PlateType"] == "COUMPOUND"

0       False
1       False
2       False
3       False
4       False
        ...  
2125    False
2126    False
2127    False
2128    False
2129    False
Name: Metadata_PlateType, Length: 2130, dtype: bool

In [118]:
paw.groupby("Metadata_Source")["Metadata_PlateType"].value_counts()

Metadata_Source  Metadata_PlateType
source_1         COMPOUND               74843
                 COMPOUND_EMPTY          5770
source_10        COMPOUND               79862
                 DMSO                    3071
                 TARGET2                 2302
source_11        COMPOUND               65663
                 TARGET2                 2684
source_13        CRISPR                 56832
                 TARGET2                 2304
source_2         COMPOUND               84032
                 TARGET2                 3828
source_3         COMPOUND               90929
                 DMSO                   14591
                 TARGET2                 9599
                 COMPOUND_EMPTY          1152
source_4         ORF                    96377
                 TARGET2                 8442
                 TARGET1                 1533
source_5         COMPOUND               75646
                 DMSO                   11904
                 TARGET2                 921

In [115]:
qq.query("has_neg == 0").Metadata_PlateType.value_counts()

Metadata_PlateType
ORF         251
CRISPR      148
POSCON8       8
COMPOUND      3
Name: count, dtype: int64

In [124]:
q.groupby(["Metadata_Source", "Metadata_PlateType"]).Metadata_Plate.nunique()

Metadata_Source  Metadata_PlateType
source_1         COMPOUND               51
source_10        COMPOUND              208
                 TARGET2                 6
source_11        COMPOUND              172
                 TARGET2                 7
source_13        CRISPR                148
                 TARGET2                 6
source_2         COMPOUND              219
                 TARGET2                10
source_3         COMPOUND              237
                 TARGET2                25
source_4         ORF                   237
                 TARGET1                 4
                 TARGET2                22
source_5         COMPOUND              197
                 TARGET2                24
source_6         COMPOUND              225
                 TARGET2                21
source_8         COMPOUND              199
                 TARGET2                 4
source_9         COMPOUND               99
                 TARGET2                 9
Name: Metadata_Pla

In [120]:
q.query("has_neg == 0").head(10)

,Metadata_Source,Metadata_Batch,Metadata_PlateType,Metadata_Plate,has_neg_compound,has_neg_orf,has_neg_crispr,has_neg_target,num_wells,has_neg
137,source_10,2021_06_28_U2OS_48_hr_run9,COMPOUND,Dest210628-162003,0,0,0,0,384,0
239,source_10,2021_08_23_U2OS_48_hr_run18,COMPOUND,Dest210823-174240,0,0,0,0,384,0
240,source_10,2021_08_23_U2OS_48_hr_run18,COMPOUND,Dest210823-174422,0,0,0,0,384,0
444,source_13,20220914_Run1,CRISPR,CP-CC9-R1-01,0,0,0,0,384,0
445,source_13,20220914_Run1,CRISPR,CP-CC9-R1-02,0,0,0,0,384,0
446,source_13,20220914_Run1,CRISPR,CP-CC9-R1-03,0,0,0,0,384,0
447,source_13,20220914_Run1,CRISPR,CP-CC9-R1-04,0,0,0,0,384,0
448,source_13,20220914_Run1,CRISPR,CP-CC9-R1-05,0,0,0,0,384,0
449,source_13,20220914_Run1,CRISPR,CP-CC9-R1-06,0,0,0,0,384,0
450,source_13,20220914_Run1,CRISPR,CP-CC9-R1-07,0,0,0,0,384,0
